In [1]:
%load_ext autoreload
%autoreload 2

import docker
import logging
from jtlutil.docker.dctl import *
import hashlib

from jtlutil.config import get_config

config = get_config('./config/config.env')

logging.basicConfig(level=logging.ERROR)
logger.setLevel(logging.DEBUG)

client = docker.DockerClient(base_url=config.SSH_URI )

print(config.SSH_URI)
ensure_network_exists(client, 'x11', is_external=False, network_type="bridge")
ensure_network_exists(client, 'jtlctl', is_external=False, network_type="bridge")
ensure_network_exists(client, 'caddy', is_external=True, network_type="bridge")

config.KST_REPORTING_URL

DEBUG:jtlutil.docker.dctl:Network 'x11' already exists.


ssh://root@64.23.135.161


DEBUG:jtlutil.docker.dctl:Network 'jtlctl' already exists.
DEBUG:jtlutil.docker.dctl:Network 'caddy' already exists.


'http://cs-spawner:8000/telem'

In [8]:
def make_servers(usernames):
    servers = []
    for username in usernames:
        nvc, pa = create_cs_pair(client, config, config.IMAGES_PYTHONCS, username)
        yield (nvc, pa)
    return servers

print('----')
for nvc, pa in make_servers(['jed','osvaldo', 'eric']):
    print(nvc.id, pa.labels['caddy'])


INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/novnc-service:v0.1.3'...


----


INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network 'caddy' : 341c37b26522cf9aaf53c4e37df9a6b94fe50cb0cc79e05d7d35d6a3f2327fb5
INFO:jtlutil.docker.dctl:Container 'jed-novnc' (b47d643d6ed75d649cfa4dab066cd7a1e24795d97609c9a575b942d884f24aba) created successfully.
INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:v0.1.5'...
INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network 'cadd

b47d643d6ed75d649cfa4dab066cd7a1e24795d97609c9a575b942d884f24aba jed.do.jointheleague.org


INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network 'caddy' : 341c37b26522cf9aaf53c4e37df9a6b94fe50cb0cc79e05d7d35d6a3f2327fb5
INFO:jtlutil.docker.dctl:Container 'osvaldo-novnc' (4413bc4bd0f4c610a987b068fadb49564207a2e6ae9ff69efda744f7b4f7e054) created successfully.
INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:v0.1.5'...
INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network '

4413bc4bd0f4c610a987b068fadb49564207a2e6ae9ff69efda744f7b4f7e054 osvaldo.do.jointheleague.org


INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network 'caddy' : 341c37b26522cf9aaf53c4e37df9a6b94fe50cb0cc79e05d7d35d6a3f2327fb5
INFO:jtlutil.docker.dctl:Container 'eric-novnc' (05528f62ac6f6fc540c46e5a138d4a84dadc183453c245dad11dcbba446ee28f) created successfully.
INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:v0.1.5'...
INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network 'cad

05528f62ac6f6fc540c46e5a138d4a84dadc183453c245dad11dcbba446ee28f eric.do.jointheleague.org


In [4]:
from time import sleep 

containers = client.containers.list(filters={"label": f"jtl"}, all=True)

for c in containers:
    print(c.id, c.status, c.labels['caddy'])
    if c.status == 'running':
        try:
            c.stop()
            c.remove()
        except Exception as e:
            print(e)

username = 'bob'

nvc, pa = create_cs_pair(client, config, config.IMAGES_PYTHONCS, username)

for i in range(10):
    print(container_status(client, pa.name))
    sleep(1)

283ef20bf2dfaa260c05cb77fcb3a9ebdadb8159b1e688c4e338983fb996f48a running bob.do.jointheleague.org
ae66ca4c70631adb44dc4c82f01f12e33ba02565a7a2bc75cca519a0f8710eb4 running bob-novnc.do.jointheleague.org


INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/novnc-service:v0.1.3'...
DEBUG:jtlutil.docker.dctl:Port bindings: None
DEBUG:jtlutil.docker.dctl:Volumes: None
INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 70fb967b471875d0782b3bfc9d4cf106fb4a653665127f127f76ba744c328a72
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : faf019272add9022b098afa197c53577b7d3f28d4e8b8afb32ae42850a7d6156
INFO:jtlutil.docker.dctl:Connecting container to network 'caddy' : 0b503a35a820d17b7a65b953195b8e2bd3c3a63bd55bc3b91cd2e4262e5d26e1
INFO:jtlutil.docker.dctl:Container 'bob-novnc' (2743709f34ec8e331f0847186f7852459d4a0f87caee37f554fa7979d7a45ea1) created successfully.
INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:v0.1.5'...
DEBUG:jtlutil.docker.dctl:Port bindings: {'8080/tcp': None}
DEBUG:jtlutil.docker.dctl:Volumes: None
INFO:jtlutil.docker.dctl:Con

running
running
running
running
running
running
running
running
running
running


In [76]:
containers = client.containers.list(filters={"label": f"jtl"}, all=True)

for c in containers:
    print(c.id, c.status, c.labels['caddy'])
    if c.status == 'running':
        try:
            c.stop()
            c.remove()
        except Exception as e:
            print(e)

        
list(make_servers(['jed']))

container_status(client, 'eric-busboom-org')

INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/novnc-service:v0.1.3'...
DEBUG:jtlutil.docker.dctl:Port bindings: None
DEBUG:jtlutil.docker.dctl:Volumes: None
INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 70fb967b471875d0782b3bfc9d4cf106fb4a653665127f127f76ba744c328a72
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : faf019272add9022b098afa197c53577b7d3f28d4e8b8afb32ae42850a7d6156
INFO:jtlutil.docker.dctl:Connecting container to network 'caddy' : 0b503a35a820d17b7a65b953195b8e2bd3c3a63bd55bc3b91cd2e4262e5d26e1
INFO:jtlutil.docker.dctl:Container 'jed-novnc' (6567efb3414b3a4a3888012ecfa2c2799d701c67b560c05d66684382cb320ec0) created successfully.
INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:v0.1.5'...
DEBUG:jtlutil.docker.dctl:Port bindings: {'8080/tcp': None}
DEBUG:jtlutil.docker.dctl:Volumes: None
INFO:jtlutil.docker.dctl:Con

c27157fa53e84e3650343ac6a80f78bc855826bd52535cdf2fa4e818c6df8e8a running jed.do.jointheleague.org
6567efb3414b3a4a3888012ecfa2c2799d701c67b560c05d66684382cb320ec0 running jed-novnc.do.jointheleague.org
b2a93daceb0a3f8be27f95825e006c978f6d61f4bae4404c3e1e7ef91ef9290c running test-student-students-jointheleague-org.do.jointheleague.org
75d528dc8e83f217b61b8d2bdb30927870d63b9efef479ca37343da989f2da34 running test-student-students-jointheleague-org-novnc.do.jointheleague.org
2ff9f5dd022f477efcf3e9691c74d4bd03f001cf22145ad7a6ecc1b8009c9b4a running eric-busboom-org.do.jointheleague.org
34aabb1d805674f6de93fef040271ab372448664899dfe4eeb8b8bae6b429186 running eric-busboom-org-novnc.do.jointheleague.org


In [77]:

containers = client.containers.list(filters={"label": f"jtl"}, all=True)

for c in containers:
    print(c.id, c.status, c.labels['caddy'])
    if c.status == 'running':
        try:
            c.stop()
            c.remove()
        except Exception as e:
            print(e)
    else:
        ... #c.start()
            

c27157fa53e84e3650343ac6a80f78bc855826bd52535cdf2fa4e818c6df8e8a running jed.do.jointheleague.org
6567efb3414b3a4a3888012ecfa2c2799d701c67b560c05d66684382cb320ec0 running jed-novnc.do.jointheleague.org
b2a93daceb0a3f8be27f95825e006c978f6d61f4bae4404c3e1e7ef91ef9290c running test-student-students-jointheleague-org.do.jointheleague.org
75d528dc8e83f217b61b8d2bdb30927870d63b9efef479ca37343da989f2da34 running test-student-students-jointheleague-org-novnc.do.jointheleague.org
2ff9f5dd022f477efcf3e9691c74d4bd03f001cf22145ad7a6ecc1b8009c9b4a running eric-busboom-org.do.jointheleague.org
34aabb1d805674f6de93fef040271ab372448664899dfe4eeb8b8bae6b429186 running eric-busboom-org-novnc.do.jointheleague.org


In [5]:
container_status(client, 'eric-busboom-org')

'non-exist'

In [44]:
id = '86fdd94433889853efc825ea18276adbd876e0fa420a84a865d2a76c5556999c'
c = client.containers.get(id)

for e in c.diff():
    print(e)
    